In [205]:
import urllib3
from bs4 import BeautifulSoup
import certifi
import pandas as pd
import pickle
import requests
import json
import numpy as np
import time

# data scrape

In [2]:
years = range(1992, 2019)

In [3]:
columns = ['japanese', 'english']

In [4]:
data = pd.DataFrame(columns=columns)

In [5]:
def getdata(year):
    url = 'http://www7a.biglobe.ne.jp/~soramimiupdate/past/' + str(year) + 'main.htm'
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    return soup

In [ ]:
for year in years:
    soup = getdata(year)
    table = 1
    while table < 200:
        if soup.select_one("#table" + str(table) + " > tr:nth-child(2) > td:nth-child(3)")==None:
            table = table + 1
        else:
            a = soup.select_one("#table" + str(table) + " > tr:nth-child(2) > td:nth-child(3)").text
            b = soup.select_one("#table" + str(table) + "> tr:nth-child(2) > td:nth-child(4)").text
            df2 = pd.DataFrame(data = [[a, b]], columns = columns)
            data= data.append(df2, ignore_index = True)
            table = table + 1

In [5]:
for i in range(data.shape[0]):
    tmp = data['japanese'][i].replace('\u3000', '')
    tmp = tmp.replace("（☆）", '')
    tmp = tmp.replace("\n", '')
    tmp = tmp.replace("\t", '')
    data['japanese'][i] = tmp

In [6]:
for i in range(data.shape[0]):
    tmp = data['english'][i].replace("\n", '')
    tmp = tmp.replace("\t", '')
    data['english'][i] = tmp

In [8]:
with open('data_refined.pickle', 'wb') as f:
    pickle.dump(data, f)

In [9]:
with open('data_refined.pickle', 'rb') as f:
    data = pickle.load(f)

In [106]:
data2 = data

In [110]:
for i in range(len(data['japanese'])):
    tmp = kanji_to_hira(data['japanese'][i])
    if(tmp):
        tmp = tmp.replace(' ', '')
        tmp = tmp.replace("\n", '')
        tmp = tmp.replace("\r", '')
        tmp = tmp.replace("〜", "ー")
        tmp = tmp.replace("！", '')
        tmp = tmp.replace("？", '')
        tmp = tmp.replace("!", '')
        tmp = tmp.replace("?", '')
        tmp = tmp.replace("・", '')
        tmp = tmp.replace(".", '')
        tmp = kata_to_hira(tmp)
        data2['japanese'][i] = tmp

In [118]:
data2.to_csv('data_preprocessed.csv')

# data load

In [9]:
with open('data_refined.pickle', 'rb') as f:
    data = pickle.load(f)

## IPAを取得

In [174]:
ipa_e = ['iː', 'ɑː', 'ɔː', 'uː', 'ɜ:', 'eɪ', 'aɪ', 'ɔi', 'aʊ', 'əʊ', 'ɪə', 'eə', 'ʊə', 'tʃ', 'dʒ', 'ə', 'ɪ', 'e', 'æ', 'ɒ', 'ʌ', 'ʊ', 'p', 'b', 't', 'd', 'k', 'g', 'f', 'v', 'θ', 'ð', 's', 'z', 'ʃ', 'ʒ', 'h', 'm', 'n', 'ŋ', 'l', 'r', 'j', 'w']


In [175]:
ipa_j = ['a', 'i', 'ɯ', 'e', 'o', 'k', 'g', 'ŋ', 's', 'ʃ', 'ʒ', 'z', 't', 'd', 'n', 'ɲ', 'h', 'ç', 'ɸ', 'b', 'p', 'm', 'j', 'r', 'w', 'ɴ', 'ː']


In [176]:
dict_jap = {'ぎゃ':'gja', 'ぎゅ':'gju', 'ぎょ': 'gjo', 'じゃ': 'dʒa', 'じゅ': 'dʒɯ','じょ':'dʒo', 'びゃ': 'bja', 'びゅ':'bjɯ' ,'びょ': 'bjo', 'ぴゃ': 'pja', 'ぴゅ': 'pjɯ', 'ぴょ':'pjo', 'きゃ':'kja', 'きゅ':'kju', 'きょ':'kjo', 'しゃ':'ʃa', 'しゅ':'ʃɯ', 'しょ':'ʃo', 'ちゃ':'tʃa', 'ちゅ':'tʃɯ', 'ちょ':'tʃo', 'にゃ':'ɲa', 'にゅ':'ɲɯ', 'にょ':'ɲo', 'ひゃ':'çja', 'ひゅ':'çjɯ', 'ひょ':'çjo', 'みゃ': 'mja', 'みゅ': 'mjɯ', 'みょ':'mjo', 'りゃ':'rja', 'りゅ': 'rjɯ', 'りょ':'rjo', 'が':'ga', 'ぎ':'gi', 'ぐ':'gɯ', 'げ':'ge', 'ご':'go', 'ざ':'dza', 'じ':'dʒi', 'ず':'dzɯ', 'ぜ':'dze', 'ぞ':'dzo', 'だ':'da', 'で':'de', 'ど':'do', 'ば':'ba', 'び':'bi', 'ぶ':'bɯ', 'べ':'be', 'ぼ':'bo', 'ぱ':'pa', 'ぴ':'pi', 'ぷ':'pɯ', 'ぺ':'pe', 'ぽ':'po', 'あ':'a', 'い':'i', 'う':'ɯ', 'え':'e', 'お':'o', 'か':'ka', 'き':'ki', 'く':'kɯ', 'け':'ke', 'こ':'ko', 'さ':'sa', 'し':'ʃi', 'す':'sɯ', 'せ':'se', 'そ':'so', 'た':'ta', 'ち':'tʃi', 'つ':'tsɯ', 'て':'te','と':'to', 'な':'na', 'に':'ɲi', 'ぬ':'nɯ', 'ね':'ne', 'の':'no', 'は':'ha', 'ひ':'çi', 'ふ':'ɸɯ', 'へ':'he', 'ほ':'ho', 'ま':'ma', 'み':'mi', 'む':'mɯ', 'め':'me', 'も':'mo', 'や':'ja', 'ゆ':'jɯ', 'よ':'yo', 'ら':'ra', 'り':'ri', 'る':'rɯ', 'れ':'re', 'ろ':'ro', 'わ':'wa', 'を':'o', 'ん':'ɴ', 'ー':'ː'}

### IPAデータ

In [201]:
N = len(data2['english'])

In [202]:
data_ipa = np.zeros((len(ipa_e) + len(ipa_j), 1000))
data_ipa = pd.DataFrame(data_ipa, index=ipa_e+ipa_j)

#### English

In [ ]:
t

In [206]:
def post_en_ipa(stre):
    time.sleep(1)
    r = requests.post("https://tophonetics.com/ja/", {"text_to_transcribe":stre})
    tmp = BeautifulSoup(r.text, 'html.parser')
    tmp = tmp.select_one("#transcr_output").text
    tmp = tmp.replace('ˈ', '')
    return tmp

In [203]:
for en in range(len(data2['english'])):
    for i in range(len(ipa_e)):
        tmp = post_en_ipa(data2['english'][en])
        print(tmp)
        if tmp.find(ipa_e[i]) != -1:
            tmp = tmp.replace('ipa_e[i]', '')
            data_ipa[en][i] = data_ipa[en][i] + 1

KeyboardInterrupt: 

#### Japanese

In [51]:
def kata_to_hira(strj):
    return "".join([chr(ord(ch) - 96) if ("ァ" <= ch <= "ヴ") else ch for ch in strj])

In [70]:
def kanji_to_hira(str):
    url = 'https://hogehoge.tk/nihongo/'
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    headers = {
        'sentence': str,
    }
    r = http.request('POST', url, headers)
    soup = BeautifulSoup(r.data, 'html.parser')
    hira = soup.select_one('body > div > div:nth-child(5) > div')
    if(hira): hira = hira.text
    return hira

In [129]:
for i in list(dict_jap.keys()):
    if data2['japanese'][0].find(i) != -1:
        print(dict_jap[i])

bɯ
i
ko
he
